# Interactive Pipeline Configuration with DAGConfigFactory

This notebook demonstrates the new interactive approach to pipeline configuration using the DAGConfigFactory.
Instead of manually creating 500+ lines of static configuration, we use a guided step-by-step process.

## Workflow Overview

1. **Define Pipeline DAG** - Create the pipeline structure
2. **Initialize DAGConfigFactory** - Set up the interactive factory
3. **Configure Base Settings** - Set shared pipeline configuration
4. **Configure Processing Settings** - Set shared processing configuration
5. **Configure Individual Steps** - Set step-specific configurations
6. **Generate Final Configurations** - Create config instances
7. **Save to JSON** - Export unified configuration file

![mods_pipeline_train_eval_calib](./tutorials/mods_end_to_end_xgboost.png)

## Environment Setup

In [1]:
import os
import json
import sys
from pathlib import Path
from datetime import datetime, date
import logging
from typing import List, Optional, Dict, Any


# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Get parent directory of current notebook
project_root = str(Path().absolute().parent)
print(f"project root {project_root}")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"add project root {project_root} into system")

project root /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines
add project root /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines into system


In [2]:
# SageMaker and SAIS imports
from sagemaker import Session
from sagemaker.workflow.pipeline_context import PipelineSession

2025-11-28 06:57:22,641 - INFO - Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2025-11-28 06:57:22,641 - INFO - NumExpr defaulting to 16 threads.
2025-11-28 06:57:23,614 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
print(f"Role: {PipelineSession().get_caller_identity_arn()}")

2025-11-28 06:57:24,007 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Role: arn:aws:iam::178936618742:role/AmazonSageMaker-ExecutionRole-Default


In [4]:
bucket = "buyer-seller-messaging-reversal"
print(f"Bucket: {bucket}")

Bucket: buyer-seller-messaging-reversal


## Step 1: Define Pipeline DAG

First, we define the pipeline structure using a DAG (Directed Acyclic Graph).
This replaces the hardcoded pipeline structure from the legacy approach.

In [5]:
from cursus.api.dag.base_dag import PipelineDAG


def create_dummy_pytorch_training_dag() -> PipelineDAG:
    """
    Create a complete end-to-end XGBoost pipeline DAG.

    This DAG represents the same workflow as the legacy demo_config.ipynb
    but in a structured, reusable format.

    Returns:
        PipelineDAG: The directed acyclic graph for the pipeline
    """
    dag = PipelineDAG()

    # Add all nodes - matching the structure from demo_config.ipynb
    dag.add_node("DummyDataLoading_training")  # Training data loading
    dag.add_node("TabularPreprocessing_training")  # Training data preprocessing
    dag.add_node("PyTorchTraining")  # XGBoost model training

    dag.add_node("DummyDataLoading_calibration")  # Dummy data load for calibration
    dag.add_node(
        "TabularPreprocessing_calibration"
    )  # Tabular preprocessing for calibration
    dag.add_node("PyTorchModelEval_calibration")  # Model evaluation step
    dag.add_node(
        "ModelCalibration_calibration"
    )  # Model calibration step with calibration variant
    dag.add_node("Package")  # Package step

    # Define dependencies - training flow
    dag.add_edge("DummyDataLoading_training", "TabularPreprocessing_training")
    dag.add_edge("TabularPreprocessing_training", "PyTorchTraining")

    # Calibration flow with Bedrock batch processing and label ruleset integration
    dag.add_edge("DummyDataLoading_calibration", "TabularPreprocessing_calibration")

    # Evaluation flow
    dag.add_edge("PyTorchTraining", "PyTorchModelEval_calibration")
    dag.add_edge(
        "TabularPreprocessing_calibration", "PyTorchModelEval_calibration"
    )  # Use labeled calibration data

    # Model calibration flow - depends on model evaluation
    dag.add_edge("PyTorchModelEval_calibration", "ModelCalibration_calibration")

    # Output flow
    dag.add_edge("ModelCalibration_calibration", "Package")
    dag.add_edge("PyTorchTraining", "Package")  # Raw model is also input to packaging

    logger.info(
        f"Created XGBoost E2E DAG with {len(dag.nodes)} nodes and {len(dag.edges)} edges"
    )
    return dag


# Create the pipeline DAG
dag = create_dummy_pytorch_training_dag()

print(f"Pipeline DAG created with {len(dag.nodes)} steps:")
for node in dag.nodes:
    print(f"  - {node}")

2025-11-28 06:57:24,549 - WARNING - Could not import constants from mods_workflow_core, using local definitions
2025-11-28 06:57:24,795 - WARNING - CradleDataLoadingStepBuilder not available. This requires secure_ai_sandbox_workflow_python_sdk package. Import error: No module named 'secure_ai_sandbox_workflow_python_sdk'
2025-11-28 06:57:24,834 - WARNING - RegistrationStepBuilder not available. This requires secure_ai_sandbox_workflow_python_sdk package. Import error: No module named 'secure_ai_sandbox_workflow_python_sdk'
2025-11-28 06:57:24,870 - INFO - Added node: DummyDataLoading_training
2025-11-28 06:57:24,870 - INFO - Added node: TabularPreprocessing_training
2025-11-28 06:57:24,870 - INFO - Added node: PyTorchTraining
2025-11-28 06:57:24,871 - INFO - Added node: DummyDataLoading_calibration
2025-11-28 06:57:24,871 - INFO - Added node: TabularPreprocessing_calibration
2025-11-28 06:57:24,872 - INFO - Added node: PyTorchModelEval_calibration
2025-11-28 06:57:24,872 - INFO - Added

Pipeline DAG created with 8 steps:
  - DummyDataLoading_training
  - TabularPreprocessing_training
  - PyTorchTraining
  - DummyDataLoading_calibration
  - TabularPreprocessing_calibration
  - PyTorchModelEval_calibration
  - ModelCalibration_calibration
  - Package


## Step 2: Initialize DAGConfigFactory

Now we initialize the DAGConfigFactory with our DAG. This will automatically:
- Map DAG nodes to configuration classes
- Set up the interactive workflow
- Prepare for step-by-step configuration

In [6]:
from cursus.api.factory.dag_config_factory import DAGConfigFactory

# Initialize the factory with our DAG
factory = DAGConfigFactory(dag)

# Get the config class mapping
config_map = factory.get_config_class_map()

print("DAG Node to Config Class Mapping:")
print("=" * 50)
for node_name, config_class in config_map.items():
    print(f"  {node_name:<35} -> {config_class.__name__}")

print(f"\nSuccessfully mapped {len(config_map)} steps to configuration classes.")

2025-11-28 06:57:24,881 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/cursus
2025-11-28 06:57:24,882 - INFO - 🔧 BuilderAutoDiscovery.__init__ - workspace_dirs: []
2025-11-28 06:57:24,883 - INFO - ✅ BuilderAutoDiscovery basic initialization complete
2025-11-28 06:57:24,883 - INFO - ✅ Registry info loaded: 41 steps
2025-11-28 06:57:24,884 - INFO - 🎉 BuilderAutoDiscovery initialization completed successfully
2025-11-28 06:57:24,884 - INFO - 🔍 ScriptAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/cursus
2025-11-28 06:57:24,884 - INFO - 🔍 ScriptAutoDiscovery.__init__ - workspace_dirs: []
2025-11-28 06:57:24,885 - INFO - 🔍 ScriptAutoDiscovery.__init__ - priority_workspace_dir: None
2025-11-28 06:57:24,885 - INFO - ✅ Registry info loaded: 41 steps
2025-11-28 06:57:24,885 - INFO - 🎉 ScriptAutoDiscovery initialization com

DAG Node to Config Class Mapping:
  DummyDataLoading_training           -> DummyDataLoadingConfig
  TabularPreprocessing_training       -> TabularPreprocessingConfig
  PyTorchTraining                     -> PyTorchTrainingConfig
  DummyDataLoading_calibration        -> DummyDataLoadingConfig
  TabularPreprocessing_calibration    -> TabularPreprocessingConfig
  PyTorchModelEval_calibration        -> PyTorchModelEvalConfig
  ModelCalibration_calibration        -> ModelCalibrationConfig
  Package                             -> PackageConfig

Successfully mapped 8 steps to configuration classes.


## Step 3: Configure Base Pipeline Settings

These settings are shared across ALL pipeline steps. Instead of repeating them
in every step configuration, we set them once here.

In [7]:
# Get base configuration requirements
base_requirements = factory.get_base_config_requirements()

print("Base Pipeline Configuration Requirements:")
print("=" * 50)
for req in base_requirements:
    marker = "*" if req["required"] else " "
    default_info = (
        f" (default: {req.get('default')})"
        if not req["required"] and "default" in req
        else ""
    )
    print(f"{marker} {req['name']:<25} ({req['type']}){default_info}")
    print(f"    {req['description']}")
    print()

Base Pipeline Configuration Requirements:
* author                    (str)
    Author or owner of the pipeline.

* bucket                    (str)
    S3 bucket name for pipeline artifacts and data.

* role                      (str)
    IAM role for pipeline execution.

* region                    (str)
    Custom region code (NA, EU, FE) for internal logic.

* service_name              (str)
    Service name for the pipeline.

* pipeline_version          (str)
    Version string for the SageMaker Pipeline.

  model_class               (str) (default: xgboost)
    Model class (e.g., XGBoost, PyTorch).

  current_date              (str) (default: PydanticUndefined)
    Current date, typically used for versioning or pathing.

  framework_version         (str) (default: 2.1.0)
    Default framework version (e.g., PyTorch).

  py_version                (str) (default: py310)
    Default Python version.

  source_dir                (Optional) (default: None)
    Common source directory fo

In [8]:
# Set up basic configuration values
region_list = ["NA", "EU", "FE"]
region_selection = 0
region = region_list[region_selection]

# Map region to AWS region
region_mapping = {"NA": "us-east-1", "EU": "eu-west-1", "FE": "us-west-2"}
aws_region = region_mapping[region]

service_name = "BuyerAbuseRnR"
pipeline_version = "0.0.1"
author = "lukexie"
model_class = "pytorch"

# Get current directory and set up paths
current_dir = Path.cwd()
package_root = Path(current_dir).resolve()
source_dir = Path("docker")
project_root_folder = "rnr_pytorch_bedrock"

# Set base configuration
factory.set_base_config(
    # Infrastructure settings
    bucket=bucket,
    role=PipelineSession().get_caller_identity_arn(),
    region=region,
    aws_region=aws_region,
    # Project identification
    author=author,
    service_name=service_name,
    pipeline_version=pipeline_version,
    model_class=model_class,
    # Framework settings
    framework_version="2.1.0",
    py_version="py310",
    source_dir=str(source_dir),
    project_root_folder=project_root_folder,
    # Date settings
    current_date=date.today().strftime("%Y-%m-%d"),
    # Enable Cache
    enable_caching=False,
    # Use Secure PyPI
    use_secure_pypi=False,
)

print("✅ Base pipeline configuration set successfully!")
print(f"   Region: {region} ({aws_region})")
print(f"   Service: {service_name}")
print(f"   Author: {author}")
print(f"   Pipeline Version: {pipeline_version}")

2025-11-28 06:57:25,257 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-11-28 06:57:25,430 - INFO - Base configuration set successfully


✅ Base pipeline configuration set successfully!
   Region: NA (us-east-1)
   Service: BuyerAbuseRnR
   Author: lukexie
   Pipeline Version: 0.0.1


## Step 4: Configure Base Processing Settings

These settings are shared across all PROCESSING steps (data loading, preprocessing, etc.)
but not training steps.

In [9]:
# Get base processing configuration requirements
processing_requirements = factory.get_base_processing_config_requirements()

if processing_requirements:
    print("Base Processing Configuration Requirements:")
    print("=" * 50)
    for req in processing_requirements:
        marker = "*" if req["required"] else " "
        default_info = (
            f" (default: {req.get('default')})"
            if not req["required"] and "default" in req
            else ""
        )
        print(f"{marker} {req['name']:<30} ({req['type']}){default_info}")
        print(f"    {req['description']}")
        print()
else:
    print("No base processing configuration required for this pipeline.")

Base Processing Configuration Requirements:
  processing_instance_count      (int) (default: 1)
    Instance count for processing jobs

  processing_volume_size         (int) (default: 500)
    Volume size for processing jobs in GB

  processing_instance_type_large (str) (default: ml.m5.4xlarge)
    Large instance type for processing step.

  processing_instance_type_small (str) (default: ml.m5.2xlarge)
    Small instance type for processing step.

  use_large_processing_instance  (bool) (default: False)
    Set to True to use large instance type, False for small instance type.

  processing_source_dir          (Optional) (default: None)
    Source directory for processing scripts. Falls back to base source_dir if not provided.

  processing_entry_point         (Optional) (default: None)
    Entry point script for processing, must be relative to source directory. Can be overridden by derived classes.

  processing_script_arguments    (Optional) (default: None)
    Optional arguments fo

In [10]:
# Set base processing configuration if needed
if processing_requirements:
    processing_source_dir = source_dir / "scripts"

    factory.set_base_processing_config(
        # Processing infrastructure
        processing_source_dir=str(processing_source_dir),
        processing_instance_type_large="ml.m5.12xlarge",
        processing_instance_type_small="ml.m5.4xlarge",
    )

    print("✅ Base processing configuration set successfully!")
    print(f"   Processing source: {processing_source_dir}")
else:
    print("✅ No base processing configuration needed.")

2025-11-28 06:57:25,444 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-28 06:57:25,444 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-28 06:57:25,445 - INFO - Base processing configuration set successfully


✅ Base processing configuration set successfully!
   Processing source: docker/scripts


## Step 5: Check Configuration Status

Let's see which steps still need configuration.

In [11]:
# Check current status
status = factory.get_configuration_status()
pending_steps = factory.get_pending_steps()

print("Configuration Status:")
print("=" * 30)
print(f"Base config set: {'✅' if status['base_config'] else '❌'}")
print(f"Processing config set: {'✅' if status['base_processing_config'] else '❌'}")
print(f"Total steps: {len(config_map)}")
print(f"Pending steps: {len(pending_steps)}")
print()

if pending_steps:
    print("Steps needing configuration:")
    for step in pending_steps:
        print(f"  - {step}")
else:
    print("✅ All steps configured!")

Configuration Status:
Base config set: ✅
Processing config set: ✅
Total steps: 8
Pending steps: 7

Steps needing configuration:
  - DummyDataLoading_training
  - TabularPreprocessing_training
  - PyTorchTraining
  - DummyDataLoading_calibration
  - TabularPreprocessing_calibration
  - PyTorchModelEval_calibration
  - ModelCalibration_calibration


## Step 6: Configure Individual Steps

Now we configure each step with its specific requirements. The factory will show us
only the fields that are unique to each step (not inherited from base configs).

### Step 6.1: Configure Training Step

This config is for **TrainingStep**. 
* It ask user to provide all necessary information to construct a **Container** and start a **Training Job**
* Ths most important information has provided in the **HyperParameter** section.


In [12]:
tab_field_list = [
    "net_conc_amt",
    "ttm_conc_amt",
    "ttm_conc_count",
    "concsi",
    "deliverable_flag",
    "undeliverable_flag",
    "unique_message_count",
    "total_ship_track_events_by_order",
    "total_unique_ship_track_events_by_order",
]

In [13]:
cat_field_list = [
    "dialogue",
    "shiptrack_event_history_by_order",
]

In [14]:
label_name = "llm_reversal_flag"
id_name = "order_id"

in_house_field_list = [
    "THREAD_ID",
    "region",
    "sender_customer_ids",
    "receipient_customer_ids",
    "entry_point_list",
    "topic_id_list",
    "subject_list",
    "shipment_ids",
    "shiptrack_tracking_id_lists_by_order",
    "shiptrack_sender_id_lists_by_order",
    "shiptrack_event_code_lists_by_order",
    "shiptrack_supplement_code_lists_by_order",
    "shiptrack_event_date_lists_by_order",
    "shiptrack_max_estimated_arrival_date_by_order",
    "shiptrack_max_estimated_delivery_date_by_order",
    "customer_id",
    "marketplace_id",
    "org",
    "concession_type",
    "abuse_type",
    "mfn_type",
    "week",
    "month",
    "order_day",
    "queued_date",
    "action_date",
    "queueid",
    "fraud_action_type_id",
    "concession_status",
    "abuse_flag",
    "concession_closed_date",
    "rr_tag",
    "reversal_reason",
    "reversal_flag",
    "reclassification_flag",
    "reclassified_segment",
    "row_num",
    "llm_status",
    "llm_error",
    "llm_category",
    "llm_confidence_score",
    "llm_key_evidence",
    "llm_reasoning",
    "llm_parse_status",
    "llm_validation_passed",
    "llm_raw_response",
    "llm_validation_error",
    id_name,
    label_name,
]

In [15]:
full_field_list = tab_field_list + cat_field_list + in_house_field_list

In [16]:
batch_size = 2
lr = 3e-05
max_epochs = 3  # 3, #15,
metric_choices = ["f1_score", "auroc"]
optimizer = "SGD"

In [17]:
# First, let's create the hyperparameters
from cursus.core.base.hyperparameters_base import ModelHyperparameters
from cursus.steps.hyperparams.hyperparameters_trimodal import TriModalHyperparameters


# Create base hyperparameters
base_hyperparameter = ModelHyperparameters(
    full_field_list=full_field_list,
    cat_field_list=cat_field_list,
    tab_field_list=tab_field_list,
    label_name=label_name,
    id_name=id_name,
    multiclass_categories=[0, 1],
    batch_size=batch_size,
    lr=lr,
    max_epochs=max_epochs,
    metric_choices=metric_choices,
    optimizer=optimizer,
)

In [18]:
model_class = "trimodal_bert"
tokenizer = "bert-base-multilingual-uncased"
primary_text_name = "dialogue"
secondary_text_name = "shiptrack_event_history_by_order"
lr_decay = 0.05
max_sen_len = 512
chunk_trancate = True
max_total_chunks = 3
momentum = 0.9
pretrained_embedding = True
reinit_layers = 2
reinit_pooler = True
run_scheduler = True
load_ckpt = False
hidden_common_dim = 100
val_check_interval = 0.25
warmup_steps = 300
weight_decay = 0
early_stop_metric = "val_loss"
early_stop_patience = 3
primary_tokenizer = secondary_tokenizer = tokenizer
text_input_ids_key = "input_ids"
text_attention_mask_key = "attention_mask"
fp16 = False
use_gradient_checkpointing = False  # False

In [19]:
# Create XGBoost hyperparameters
trimodal_hyperparams = TriModalHyperparameters.from_base_hyperparam(
    base_hyperparameter,
    model_class=model_class,
    tokenizer=tokenizer,
    primary_text_name=primary_text_name,
    secondary_text_name=secondary_text_name,
    text_input_ids_key=text_input_ids_key,
    text_attention_mask_key=text_attention_mask_key,
    momentum=momentum,
    lr_decay=lr_decay,
    weight_decay=weight_decay,
    max_sen_len=max_sen_len,
    chunk_trancate=chunk_trancate,
    max_total_chunks=max_total_chunks,
    reinit_layers=reinit_layers,
    reinit_pooler=reinit_pooler,
    run_scheduler=run_scheduler,
    load_ckpt=load_ckpt,
    hidden_common_dim=hidden_common_dim,
    warmup_steps=warmup_steps,
    val_check_interval=val_check_interval,
    early_stop_metric=early_stop_metric,
    early_stop_patience=early_stop_patience,
    fp16=fp16,
    use_gradient_checkpointing=use_gradient_checkpointing,
)

print("✅ Hyperparameters created")
print(
    f"   Features: {len(full_field_list)} total, {len(tab_field_list)} numerical, {len(cat_field_list)} categorical"
)
print(f"   XGBoost rounds: {trimodal_hyperparams.max_epochs}")

✅ Hyperparameters created
   Features: 60 total, 9 numerical, 2 categorical
   XGBoost rounds: 3


In [20]:
instance_type_list = [
    "ml.m5.4xlarge",
    "ml.m5.12xlarge",
    "ml.p3.16xlarge",
    "ml.g4dn.16xlarge",
    "ml.g5.12xlarge",
    "ml.g5.16xlarge",
]

In [21]:
instance_select = -4  # -2 #-1
instance_type_list[instance_select]

'ml.p3.16xlarge'

In [22]:
# Configure XGBoost training
if "PyTorchTraining" in pending_steps:
    step_name = "PyTorchTraining"
    training_volume_size = 800
    factory.set_step_config(
        step_name,
        training_instance_type=instance_type_list[instance_select],
        training_entry_point="pytorch_training.py",
        training_volume_size=training_volume_size,
    )
    print(f"✅ {step_name} configured")
    print(f"   Instance type: {instance_type_list[instance_select]}")
    print(f"   Volume size: {training_volume_size} GB")

2025-11-28 06:57:25,520 - INFO - ✅ PyTorchTraining configured successfully using PyTorchTrainingConfig


✅ PyTorchTraining configured
   Instance type: ml.p3.16xlarge
   Volume size: 800 GB


### Step 6.4: Configure Preprocessing Steps

In [23]:
# Configure training preprocessing
if "TabularPreprocessing_training" in pending_steps:
    step_name = "TabularPreprocessing_training"

    factory.set_step_config(
        step_name,
        job_type="training",
        label_name=label_name,
        processing_entry_point="tabular_preprocessing.py",
        use_large_processing_instance=True,
        output_format="Parquet",
    )
    print(f"✅ {step_name} configured")

2025-11-28 06:57:25,526 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,527 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,527 - INFO - ✅ TabularPreprocessing_training configured successfully using TabularPreprocessingConfig


✅ TabularPreprocessing_training configured


In [24]:
# Configure training preprocessing
if "TabularPreprocessing_calibration" in pending_steps:
    step_name = "TabularPreprocessing_calibration"

    factory.set_step_config(
        step_name,
        job_type="calibration",
        label_name=None,
        processing_entry_point="tabular_preprocessing.py",
        use_large_processing_instance=True,
        output_format="Parquet",
    )
    print(f"✅ {step_name} configured")

2025-11-28 06:57:25,532 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,533 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,533 - INFO - ✅ TabularPreprocessing_calibration configured successfully using TabularPreprocessingConfig


✅ TabularPreprocessing_calibration configured


### Step 6.4: Configure Remaining Steps

**USER INPUT BLOCK**: Fill in the essential fields for each remaining step.
The factory has identified the required fields for each step.

In [25]:
# Get current pending steps
current_pending = factory.get_pending_steps()

print("Remaining steps to configure:")
print("=" * 40)

for step_name in current_pending:
    requirements = factory.get_step_requirements(step_name)
    essential_reqs = [req for req in requirements if req["required"]]

    print(f"\n{step_name}:")
    print(f"  Essential fields ({len(essential_reqs)}):")
    for req in essential_reqs:
        print(f"    * {req['name']} ({req['type']}) - {req['description']}")

    if len(requirements) > len(essential_reqs):
        optional_count = len(requirements) - len(essential_reqs)
        print(f"  Optional fields: {optional_count}")

Remaining steps to configure:

DummyDataLoading_training:
  Essential fields (1):
    * data_source (Union) - Local directory path or S3 URI where the input data is stored. Examples: '/path/to/local/data' or 's3://bucket/path/to/data'
  Optional fields: 6

DummyDataLoading_calibration:
  Essential fields (1):
    * data_source (Union) - Local directory path or S3 URI where the input data is stored. Examples: '/path/to/local/data' or 's3://bucket/path/to/data'
  Optional fields: 6

PyTorchModelEval_calibration:
  Essential fields (2):
    * id_name (str) - Name of the ID field in the dataset (required for evaluation).
    * label_name (str) - Name of the label field in the dataset (required for evaluation).
  Optional fields: 8

ModelCalibration_calibration:
  Essential fields (1):
    * label_field (str) - Name of the label column
  Optional fields: 12


In [26]:
id_name

'order_id'

In [27]:
label_name

'llm_reversal_flag'

In [28]:
# Configure dummy data loading
if "DummyDataLoading_training" in pending_steps:
    step_name = "DummyDataLoading_training"

    data_source = "s3://buyer-seller-messaging-reversal/pipeline/lukexie-BuyerAbuseRnR-pytorch-NA/20251111035543/labelrulesetexecution/processed_data/"

    factory.set_step_config(
        step_name,
        job_type="training",
        data_source=data_source,
        processing_entry_point="dummy_data_loading.py",
        use_large_processing_instance=True,
        write_data_shards=True,
        output_format="PARQUET",
    )
    print(f"✅ {step_name} configured")

2025-11-28 06:57:25,559 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,560 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,560 - INFO - ✅ DummyDataLoading_training configured successfully using DummyDataLoadingConfig


✅ DummyDataLoading_training configured


In [29]:
# Configure dummy data loading
if "DummyDataLoading_calibration" in pending_steps:
    step_name = "DummyDataLoading_calibration"

    data_source = "s3://buyer-seller-messaging-reversal/pipeline/lukexie-BuyerAbuseRnR-pytorch-NA/20251111035543/labelrulesetexecution/processed_data/test"

    factory.set_step_config(
        step_name,
        job_type="calibration",
        data_source=data_source,
        processing_entry_point="dummy_data_loading.py",
        use_large_processing_instance=True,
        write_data_shards=True,
        output_format="PARQUET",
    )
    print(f"✅ {step_name} configured")

2025-11-28 06:57:25,566 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,566 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,567 - INFO - ✅ DummyDataLoading_calibration configured successfully using DummyDataLoadingConfig


✅ DummyDataLoading_calibration configured


In [30]:
# Configure Model Evaluation
if "PyTorchModelEval_calibration" in current_pending:
    step_name = "PyTorchModelEval_calibration"
    factory.set_step_config(
        step_name,
        job_type="calibration",
        processing_entry_point="pytorch_model_eval.py",
        id_name=id_name,
        label_name=label_name,
        processing_source_dir=str(source_dir),
        processing_instance_type_large="ml.g5.12xlarge",
        use_large_processing_instance=True,
        processing_framework_version="2.1.2",
        py_version="py310",
    )
    print(f"✅ {step_name} configured")

2025-11-28 06:57:25,572 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-28 06:57:25,572 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-28 06:57:25,573 - INFO - ✅ PyTorchModelEval_calibration configured successfully using PyTorchModelEvalConfig


✅ PyTorchModelEval_calibration configured


In [31]:
# Configure Model Calibration
if "ModelCalibration_calibration" in current_pending:
    factory.set_step_config(
        "ModelCalibration_calibration",
        label_field=label_name,
        processing_entry_point="model_calibration.py",
        score_field="prob_class_1",
        is_binary=True,
        num_classes=2,
        score_field_prefix="prob_class_",
        multiclass_categories=[0, 1],
    )
    print(f"✅ ModelCalibration_calibration configured")

2025-11-28 06:57:25,578 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,579 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,579 - INFO - ✅ ModelCalibration_calibration configured successfully using ModelCalibrationConfig


✅ ModelCalibration_calibration configured


In [32]:
# Configure Model Calibration
if "Payload" in current_pending:
    factory.set_step_config(
        "Payload",
        processing_entry_point="payload.py",
        # =================================
        expected_tps=10,
        max_latency_in_millisecond=800,
    )
    print(f"✅ Payload configured")

## Step 7: Generate Final Configurations

Now that all steps are configured, we can generate the final configuration instances.
The factory will validate that all essential fields are provided and create the config objects.

In [33]:
# Check final status
final_status = factory.get_configuration_status()
final_pending = factory.get_pending_steps()

print("Final Configuration Status:")
print("=" * 40)
print(f"Base config: {'✅' if final_status['base_config'] else '❌'}")
print(f"Processing config: {'✅' if final_status['base_processing_config'] else '❌'}")
print(f"Pending steps: {len(final_pending)}")

if final_pending:
    print("\nStill pending:")
    for step in final_pending:
        print(f"  - {step}")
    print("\n⚠️  Please configure remaining steps before generating configs.")
else:
    print("\n✅ All steps configured! Ready to generate configurations.")

Final Configuration Status:
Base config: ✅
Processing config: ✅
Pending steps: 0

✅ All steps configured! Ready to generate configurations.


In [34]:
# Generate final configurations
if not final_pending:
    try:
        print("Generating final configurations...")
        configs = factory.generate_all_configs()

        print(f"\n✅ Successfully generated {len(configs)} configuration instances:")
        for i, config in enumerate(configs, 1):
            print(f"  {i:2d}. {config.__class__.__name__}")

        print("\n🎉 Configuration generation complete!")

    except Exception as e:
        print(f"\n❌ Configuration generation failed: {e}")
        print("\nPlease check that all required fields are provided.")
        configs = None
else:
    print("\n⚠️  Cannot generate configs - some steps are still pending configuration.")
    configs = None

2025-11-28 06:57:25,598 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,599 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker/scripts
2025-11-28 06:57:25,599 - INFO - ✅ Package auto-configured successfully (only tier 2+ fields)
2025-11-28 06:57:25,599 - INFO - ✅ Auto-configured 1 steps with only tier 2+ fields
2025-11-28 06:57:25,600 - INFO - ✅ Returning 8 pre-validated configuration instances


Generating final configurations...

✅ Successfully generated 8 configuration instances:
   1. PyTorchTrainingConfig
   2. TabularPreprocessingConfig
   3. TabularPreprocessingConfig
   4. DummyDataLoadingConfig
   5. DummyDataLoadingConfig
   6. PyTorchModelEvalConfig
   7. ModelCalibrationConfig
   8. PackageConfig

🎉 Configuration generation complete!


In [35]:
len(configs)

8

## Step 8: Save to JSON

Finally, we save the generated configurations to a unified JSON file using the existing
`merge_and_save_configs` utility. This creates the same format as the legacy approach
but with much less effort!

In [36]:
if configs:
    # Set up output directory and filename
    MODEL_CLASS = "pytorch"
    service_name = "BuyerAbuseRnR"

    config_dir = Path(current_dir) / "pipeline_config"
    config_dir.mkdir(parents=True, exist_ok=True)

    config_file_name = f"config.json"
    config_path = config_dir / config_file_name

    print(f"Saving configurations to: {config_path}")

    # Use the existing merge_and_save_configs utility
    from cursus.steps.configs.utils import merge_and_save_configs

    try:
        merged_config = merge_and_save_configs(configs, str(config_path))

        print(f"\n✅ Configuration saved successfully!")
        print(f"   File: {config_path}")
        print(f"   Size: {config_path.stat().st_size / 1024:.1f} KB")

        # Also save hyperparameters separately (for compatibility)
        hyperparam_path = source_dir / "hyperparams" / f"hyperparameters.json"
        with open(hyperparam_path, "w") as f:
            json.dump(trimodal_hyperparams.model_dump(), f, indent=2, sort_keys=True)

        print(f"   Hyperparameters: {hyperparam_path}")

        print(f"\n🎉 Interactive configuration complete!")
        print(f"\n📊 Comparison with legacy approach:")
        print(f"   Legacy: 500+ lines of manual configuration")
        print(f"   Interactive: Guided step-by-step process")
        print(f"   Time saved: ~20-25 minutes")
        print(f"   Error reduction: Validation at each step")

    except Exception as e:
        print(f"\n❌ Failed to save configurations: {e}")

else:
    print("\n⚠️  No configurations to save. Please generate configs first.")

2025-11-28 06:57:25,612 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-28 06:57:25,612 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-28 06:57:25,616 - INFO - Merging and saving 8 configs to /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-28 06:57:25,617 - INFO - Collecting field information for 8 configs (7 processing configs)
2025-11-28 06:57:25,621 - INFO - Collected information for 66 unique fields
2025-11-28 06:57:25,623 - INFO - Efficient algorithm identified 15 shared fields
2025-11-28 06:57:25,625 - INFO - Populated specific fields for 8 configs
2025-11-28 06:57:25,626 - INFO - Shared fields: 15
2025-11-28 06:57:25,626 - INFO -

Saving configurations to: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json

✅ Configuration saved successfully!
   File: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
   Size: 24.7 KB
   Hyperparameters: docker/hyperparams/hyperparameters.json

🎉 Interactive configuration complete!

📊 Comparison with legacy approach:
   Legacy: 500+ lines of manual configuration
   Interactive: Guided step-by-step process
   Time saved: ~20-25 minutes
   Error reduction: Validation at each step


### Test if we can load it

In [37]:
from cursus.steps.configs.config_dummy_data_loading_step import DummyDataLoadingConfig
from cursus.steps.configs.config_tabular_preprocessing_step import (
    TabularPreprocessingConfig,
)
from cursus.steps.configs.config_pytorch_training_step import PyTorchTrainingConfig
from cursus.steps.configs.config_pytorch_model_eval_step import PyTorchModelEvalConfig
from cursus.steps.configs.config_model_calibration_step import ModelCalibrationConfig
from cursus.steps.configs.config_package_step import PackageConfig
from cursus.steps.configs.config_payload_step import PayloadConfig

In [38]:
from cursus.steps.configs.utils import load_configs

In [39]:
CONFIG_CLASSES = {
    "DummyDataLoadingConfig": DummyDataLoadingConfig,
    "TabularPreprocessingConfig": TabularPreprocessingConfig,
    "PyTorchTrainingConfig": PyTorchTrainingConfig,
    "PyTorchModelEvalConfig": PyTorchModelEvalConfig,
    "ModelCalibrationConfig": ModelCalibrationConfig,
    "PackageConfig": PackageConfig,
    "PayloadConfig": PayloadConfig,
}

In [40]:
loaded_configs = load_configs(str(config_path), CONFIG_CLASSES)

2025-11-28 06:57:25,651 - INFO - Loading configs from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-28 06:57:25,652 - INFO - Loading configuration from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-28 06:57:25,653 - INFO - Successfully loaded configuration from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-28 06:57:25,653 - INFO - Successfully loaded configs from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json with 8 specific configs
2025-11-28 06:57:25,654 - INFO - Creating additional config instance for DummyDataLoading_calibration (DummyDataLoadingConfig)
2025-11-28 06:57:25,654 - INFO - Package location discovery succeeded (bundled): /home/ec2-us

In [41]:
len(loaded_configs)

8

## Summary

This notebook demonstrates the **DAGConfigFactory** approach to pipeline configuration:

### ✅ **Benefits Achieved**

1. **Reduced Complexity**: From 500+ lines of manual config to guided workflow
2. **Base Config Inheritance**: Set common fields once, inherit everywhere
3. **Step-by-Step Guidance**: Clear requirements for each configuration step
4. **Validation**: Comprehensive validation prevents configuration errors
5. **Reusable DAG**: Pipeline structure defined once, reused across environments

### 🔄 **Workflow Comparison**

| Aspect | Legacy Approach | Interactive Approach |
|--------|----------------|---------------------|
| **Lines of Code** | 500+ manual lines | Guided step-by-step |
| **Time Required** | 30+ minutes | 10-15 minutes |
| **Error Rate** | High (manual entry) | Low (validation) |
| **Reusability** | Copy-paste heavy | DAG-driven |
| **Maintenance** | Manual updates | Automatic inheritance |

### 🚀 **Next Steps**

The generated configuration file can now be used with the existing pipeline compiler:

```python
# Use with pipeline compiler (from demo_pipeline.ipynb)
from cursus.core.compiler.dag_compiler import PipelineDAGCompiler

dag_compiler = PipelineDAGCompiler(
    config_path=config_path,
    sagemaker_session=pipeline_session,
    role=role
)

# Compile DAG to pipeline
template_pipeline, report = dag_compiler.compile_with_report(dag=dag)
```

The interactive configuration approach transforms the user experience from complex manual setup to an intuitive, guided workflow while maintaining full compatibility with the existing cursus infrastructure.